In [2]:
# ==============================================================================
# 1. SETUP: Install Required Libraries
# ==============================================================================
# Using pyngrok to expose our local server to the internet for webhook callbacks.
%pip install twilio flask python-dotenv pyngrok --quiet

Note: you may need to restart the kernel to use updated packages.


In [6]:
# ==============================================================================
# 2. CONFIGURATION: Load Environment Variables and Initialize Twilio Client
# ==============================================================================
import os
import threading
import time
from dotenv import load_dotenv
from twilio.rest import Client

# Load environment variables from a .env file in the same directory
load_dotenv()

# Fetch credentials from environment variables
ACCOUNT_SID = os.getenv('TWILIO_ACCOUNT_SID')
AUTH_TOKEN = os.getenv('TWILIO_AUTH_TOKEN')
FROM_NUMBER = os.getenv('TWILIO_PHONE_NUMBER')  # Your Twilio number for making calls
TO_NUMBER = os.getenv('INCOMING_PHONE_NUMBER') # The number you will be calling
NGROK_DOMAIN = os.getenv('NGROK_DOMAIN')  # Your ngrok domain
PORT = 5000  # Port for the Flask server

# Validate that all required variables are set
if not all([ACCOUNT_SID, AUTH_TOKEN, FROM_NUMBER, TO_NUMBER]):
    print("ERROR: Missing one or more required environment variables.")
    print("   Please create a .env file with the following content:")
    print("   TWILIO_ACCOUNT_SID=ACxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
    print("   TWILIO_AUTH_TOKEN=your_auth_token_here")
    print("   TWILIO_PHONE_NUMBER=+15017122661")
    print("   INCOMING_PHONE_NUMBER=+15558675310")
    print("   NGROK_DOMAIN=your-ngrok-domain.ngrok.io")
else:
    client = Client(ACCOUNT_SID, AUTH_TOKEN)
    print("Twilio client initialized successfully.")
    print(f"   Account SID: {ACCOUNT_SID[:5]}...")
    print(f"   Outbound Number: {FROM_NUMBER}")
    print(f"   Destination Number: {TO_NUMBER}")
    print(f"   Ngrok Domain: {NGROK_DOMAIN}")

Twilio client initialized successfully.
   Account SID: ACdf2...
   Outbound Number: +551150266619
   Destination Number: +551148583195
   Ngrok Domain: owlbank.ngrok.io


In [ ]:
# ==============================================================================
# 3. WEBHOOK SERVER: Setup Flask and Expose via Ngrok
# ==============================================================================
from flask import Flask, request
from twilio.twiml.voice_response import VoiceResponse
from pyngrok import ngrok

# --- Flask App Definition ---
app = Flask(__name__)

@app.route("/webhook", methods=['POST'])
def handle_webhook():
    """Handles incoming Twilio webhook requests for AMD status updates."""
    print("\n" + "="*60)
    print("--- AMD WEBHOOK RECEIVED ---")
    print(f"Timestamp: {time.strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Extract AMD-specific parameters
    call_sid = request.form.get('CallSid', 'N/A')
    call_status = request.form.get('CallStatus', 'N/A')
    answered_by = request.form.get('AnsweredBy', 'N/A')
    
    print(f"Call SID: {call_sid}")
    print(f"Call Status: {call_status}")
    print(f"AMD Result: {answered_by}")
    
    # Check for additional AMD data
    if 'MachineDetectionDuration' in request.form:
        print(f"AMD Duration: {request.form.get('MachineDetectionDuration')}ms")
    
    print("="*60)
    print("Full Webhook Data:")
    for key, value in request.form.items():
        print(f"  {key}: {value}")
    print("="*60 + "\n")
    
    return str(VoiceResponse()), 200

@app.route("/silent", methods=['POST'])
def handle_silent():
    """Returns TwiML that keeps caller silent to listen for callee response."""
    response = VoiceResponse()
    # Keep caller silent for 60 seconds to allow AMD detection
    response.pause(length=60)
    return str(response)

# --- Server and Ngrok Tunnel Control ---
def run_app():
    app.run(host='0.0.0.0', port=PORT, debug=False)

def start_server_and_ngrok():
    """Starts the Flask server in a thread and creates a public ngrok tunnel."""
    # Start Flask server in a background thread
    flask_thread = threading.Thread(target=run_app, daemon=True)
    flask_thread.start()
    time.sleep(1)

    # Start ngrok tunnel
    try:
        public_url = ngrok.connect(PORT)
        print("\nWebhook server is running and exposed via ngrok.")
        print(f"   Public URL: {public_url}")
        return public_url
    except Exception as e:
        print(f"ERROR: Could not start ngrok. Please ensure ngrok is installed and configured.")
        print(f"   Error details: {e}")
        return None

def stop_server_and_ngrok(public_url):
    """Shuts down the specified ngrok tunnel."""
    if not public_url:
        print("\nNo active tunnel to shut down.")
        return
        
    print("\nShutting down ngrok tunnel...")
    ngrok.disconnect(public_url)
    print("   Tunnel closed.")

# --- Start the server and get the public URL for our demo ---
PUBLIC_URL = start_server_and_ngrok()

# Set webhook base URL for configurations
if PUBLIC_URL:
    WEBHOOK_BASE_URL = PUBLIC_URL
elif NGROK_DOMAIN:
    WEBHOOK_BASE_URL = f"https://{NGROK_DOMAIN}"
else:
    WEBHOOK_BASE_URL = "https://your-ngrok-domain.ngrok.io"
    print("WARNING: Using placeholder webhook URL. Please update NGROK_DOMAIN in .env")

print(f"Webhook Base URL: {WEBHOOK_BASE_URL}")

In [7]:
# ==============================================================================
# 4. AMD CONFIGURATIONS: Define Different Testing Scenarios
# ==============================================================================

# AMD configurations for different testing scenarios
AMD_CONFIGURATIONS = {
    "residential_fast": {
        "name": "Residential/Mobile - Fast Detection",
        "description": "Optimized for personal phones with short greetings",
        "params": {
            "machine_detection": "Enable",
            "machine_detection_timeout": 15,
            "machine_detection_speech_threshold": 1800,
            "machine_detection_speech_end_threshold": 1400,
            "machine_detection_silence_timeout": 3000,
            "async_amd_status_callback": f"{WEBHOOK_BASE_URL}/webhook",
            "async_amd_status_callback_method": "POST",
            "url": f"{WEBHOOK_BASE_URL}/silent"  # Keep caller silent
        }
    },
    "business_standard": {
        "name": "Business - Standard Detection", 
        "description": "Optimized for business phones with longer greetings",
        "params": {
            "machine_detection": "Enable",
            "machine_detection_timeout": 20,
            "machine_detection_speech_threshold": 2400,
            "machine_detection_speech_end_threshold": 1500,
            "machine_detection_silence_timeout": 4000,
            "async_amd_status_callback": f"{WEBHOOK_BASE_URL}/webhook",
            "async_amd_status_callback_method": "POST",
            "url": f"{WEBHOOK_BASE_URL}/silent"
        }
    },
    "voicemail_drop": {
        "name": "Voicemail Message Drop",
        "description": "Waits for complete voicemail greeting to end (DetectMessageEnd)",
        "params": {
            "machine_detection": "DetectMessageEnd",
            "machine_detection_timeout": 45,
            "machine_detection_speech_threshold": 3000,
            "machine_detection_speech_end_threshold": 1200,
            "machine_detection_silence_timeout": 5000,
            "async_amd_status_callback": f"{WEBHOOK_BASE_URL}/webhook",
            "async_amd_status_callback_method": "POST",
            "url": f"{WEBHOOK_BASE_URL}/silent"
        }
    },
    "conservative": {
        "name": "Conservative - High Accuracy",
        "description": "Longer timeouts for higher accuracy, fewer 'unknown' results",
        "params": {
            "machine_detection": "Enable", 
            "machine_detection_timeout": 25,
            "machine_detection_speech_threshold": 2400,
            "machine_detection_speech_end_threshold": 1200,
            "machine_detection_silence_timeout": 5000,
            "async_amd_status_callback": f"{WEBHOOK_BASE_URL}/webhook",
            "async_amd_status_callback_method": "POST",
            "url": f"{WEBHOOK_BASE_URL}/silent"
        }
    },
    "aggressive": {
        "name": "Aggressive - Fast Response",
        "description": "Shorter timeouts for faster detection, may increase 'unknown' results",
        "params": {
            "machine_detection": "Enable",
            "machine_detection_timeout": 8,
            "machine_detection_speech_threshold": 1500,
            "machine_detection_speech_end_threshold": 1000,
            "machine_detection_silence_timeout": 2500,
            "async_amd_status_callback": f"{WEBHOOK_BASE_URL}/webhook",
            "async_amd_status_callback_method": "POST",
            "url": f"{WEBHOOK_BASE_URL}/silent"
        }
    },
    "custom_tuning": {
        "name": "Custom Fine-Tuning",
        "description": "Template for custom parameter fine-tuning",
        "params": {
            "machine_detection": "Enable",
            "machine_detection_timeout": 20,
            "machine_detection_speech_threshold": 2000,
            "machine_detection_speech_end_threshold": 1200,
            "machine_detection_silence_timeout": 4000,
            "async_amd_status_callback": f"{WEBHOOK_BASE_URL}/webhook",
            "async_amd_status_callback_method": "POST",
            "url": f"{WEBHOOK_BASE_URL}/silent"
        }
    }
}

print("Twilio AMD Testing Configurations:")
print("=" * 60)
for key, config in AMD_CONFIGURATIONS.items():
    print(f"'{key}': {config['name']}")
    print(f"   {config['description']}")
    params = config['params']
    print(f"   Detection: {params['machine_detection']}")
    print(f"   Timeout: {params['machine_detection_timeout']}s")
    print(f"   Speech Threshold: {params['machine_detection_speech_threshold']}ms")
    print(f"   Speech End Threshold: {params['machine_detection_speech_end_threshold']}ms")
    print(f"   Silence Timeout: {params['machine_detection_silence_timeout']}ms")
    print()

Twilio AMD Testing Configurations:
'residential_fast': Residential/Mobile - Fast Detection
   Optimized for personal phones with short greetings
   Detection: Enable
   Timeout: 15s
   Speech Threshold: 1800ms
   Speech End Threshold: 1400ms
   Silence Timeout: 3000ms

'business_standard': Business - Standard Detection
   Optimized for business phones with longer greetings
   Detection: Enable
   Timeout: 20s
   Speech Threshold: 2400ms
   Speech End Threshold: 1500ms
   Silence Timeout: 4000ms

'voicemail_drop': Voicemail Message Drop
   Waits for complete voicemail greeting to end (DetectMessageEnd)
   Detection: DetectMessageEnd
   Timeout: 45s
   Speech Threshold: 3000ms
   Speech End Threshold: 1200ms
   Silence Timeout: 5000ms

'conservative': Conservative - High Accuracy
   Longer timeouts for higher accuracy, fewer 'unknown' results
   Detection: Enable
   Timeout: 25s
   Speech Threshold: 2400ms
   Speech End Threshold: 1200ms
   Silence Timeout: 5000ms

'aggressive': Aggressi

In [11]:
# ==============================================================================
# 5. AMD TESTING FUNCTIONS
# ==============================================================================

def make_amd_call(config_key, from_number=None, to_number=None):
    """Make an AMD call with specified configuration"""
    
    if config_key not in AMD_CONFIGURATIONS:
        print(f"Invalid configuration key: {config_key}")
        print(f"Available configurations: {list(AMD_CONFIGURATIONS.keys())}")
        return None
    
    config = AMD_CONFIGURATIONS[config_key]
    from_num = from_number or FROM_NUMBER
    to_num = to_number or TO_NUMBER
    
    if not from_num or not to_num:
        print("Error: Phone numbers not configured")
        return None
    
    print(f"\nMaking AMD call with: {config['name']}")
    print(f"Description: {config['description']}")
    print(f"From: {from_num}")
    print(f"To: {to_num}")
    
    # Display AMD parameters
    params = config['params']
    print(f"\nAMD Parameters:")
    print(f"  Detection Type: {params['machine_detection']}")
    print(f"  Timeout: {params['machine_detection_timeout']} seconds")
    print(f"  Speech Threshold: {params['machine_detection_speech_threshold']} ms")
    print(f"  Speech End Threshold: {params['machine_detection_speech_end_threshold']} ms")
    print(f"  Silence Timeout: {params['machine_detection_silence_timeout']} ms")
    print(f"  Webhook URL: {params['async_amd_status_callback']}")
    print(f"  TwiML URL: {params['url']} (caller stays silent)")
    
    try:
        # Prepare call parameters
        call_params = {
            'to': to_num,
            'from_': from_num,
            **params
        }
        
        # Make the call
        call = client.calls.create(**call_params)
        
        print(f"\nCall initiated successfully!")
        print(f"Call SID: {call.sid}")
        print(f"Status: {call.status}")
        print(f"Direction: {call.direction}")
        print(f"Created: {call.date_created}")
        
        return call
    
    except Exception as e:
        print(f"Error making call: {str(e)}")
        return None

def fetch_call_amd_results(call_sid):
    """Fetch and display AMD results from a call"""
    try:
        call = client.calls(call_sid).fetch()
        
        print(f"\nCall AMD Results for {call_sid}:")
        print("-" * 50)
        print(f"Call Status: {call.status}")
        print(f"Duration: {call.duration} seconds" if call.duration else "Duration: N/A")
        print(f"Start Time: {call.start_time}")
        print(f"End Time: {call.end_time}" if call.end_time else "End Time: N/A")
        
        # Check for AMD-specific attributes
        amd_attributes = [
            'answered_by', 'machine_detection_duration', 
            'machine_detection_silence_timeout', 'machine_detection_speech_threshold',
            'machine_detection_speech_end_threshold', 'machine_detection_timeout'
        ]
        
        amd_results = {}
        for attr in amd_attributes:
            if hasattr(call, attr):
                value = getattr(call, attr)
                if value is not None:
                    amd_results[attr] = value
        
        if amd_results:
            print(f"\nAMD Results:")
            for key, value in amd_results.items():
                formatted_key = key.replace('_', ' ').title()
                print(f"  {formatted_key}: {value}")
        else:
            print("\nNo AMD results found in call object")
            print("AMD results should appear in webhook callbacks")
        
        return call
        
    except Exception as e:
        print(f"Error fetching call: {str(e)}")
        return None

def monitor_call_with_amd_results(call_sid, max_wait_time=60):
    """Enhanced monitoring that checks for AMD results"""
    print(f"\nMonitoring call with AMD results: {call_sid}")
    print("-" * 50)
    
    start_time = time.time()
    last_status = None
    amd_result_shown = False
    
    while (time.time() - start_time) < max_wait_time:
        try:
            call = client.calls(call_sid).fetch()
            
            if call.status != last_status:
                print(f"{time.strftime('%H:%M:%S')} - Status: {call.status}")
                last_status = call.status
            
            # Check for AMD results in call object
            if hasattr(call, 'answered_by') and call.answered_by and not amd_result_shown:
                print(f"{time.strftime('%H:%M:%S')} - AMD Result from Call Object: {call.answered_by}")
                amd_result_shown = True
                
                # Show additional AMD details if available
                if hasattr(call, 'machine_detection_duration') and call.machine_detection_duration:
                    print(f"{time.strftime('%H:%M:%S')} - AMD Duration: {call.machine_detection_duration}ms")
            
            if call.status in ['completed', 'failed', 'canceled', 'busy', 'no-answer']:
                print(f"{time.strftime('%H:%M:%S')} - Call ended with status: {call.status}")
                if hasattr(call, 'duration') and call.duration:
                    print(f"Total Duration: {call.duration} seconds")
                
                # Final AMD results check
                print(f"\nFinal AMD Results Check:")
                fetch_call_amd_results(call.sid)
                break
            
            time.sleep(2)
            
        except Exception as e:
            print(f"Error monitoring call: {str(e)}")
            break
    
    print("Monitoring complete\n")

def create_custom_config(name, description, detection_type="Enable", timeout=20, 
                        speech_threshold=2000, speech_end_threshold=1200, 
                        silence_timeout=4000):
    """Create a custom AMD configuration for fine-tuning"""
    
    custom_config = {
        "name": name,
        "description": description,
        "params": {
            "machine_detection": detection_type,
            "machine_detection_timeout": timeout,
            "machine_detection_speech_threshold": speech_threshold,
            "machine_detection_speech_end_threshold": speech_end_threshold,
            "machine_detection_silence_timeout": silence_timeout,
            "async_amd_status_callback": f"{WEBHOOK_BASE_URL}/webhook",
            "async_amd_status_callback_method": "POST",
            "url": f"{WEBHOOK_BASE_URL}/silent"
        }
    }
    
    # Add to configurations with a unique key
    config_key = f"custom_{int(time.time())}"
    AMD_CONFIGURATIONS[config_key] = custom_config
    
    print(f"Created custom configuration: {config_key}")
    print(f"Name: {name}")
    print(f"Description: {description}")
    print(f"Parameters: Timeout={timeout}s, Speech={speech_threshold}ms, "
          f"End={speech_end_threshold}ms, Silence={silence_timeout}ms")
    
    return config_key

def batch_test_all_configurations():
    """Test all AMD configurations in sequence"""
    print("\n" + "="*60)
    print("BATCH TESTING ALL AMD CONFIGURATIONS")
    print("="*60)
    
    if not FROM_NUMBER or not TO_NUMBER:
        print("Error: Please configure phone numbers in environment variables")
        return
    
    configs_to_test = list(AMD_CONFIGURATIONS.keys())
    results = {}
    
    print(f"Will test {len(configs_to_test)} configurations:")
    for config_key in configs_to_test:
        config = AMD_CONFIGURATIONS[config_key]
        print(f"  - {config_key}: {config['name']}")
    
    confirm = input(f"\nProceed with batch testing? This will make {len(configs_to_test)} calls (y/n): ").strip().lower()
    
    if confirm != 'y':
        print("Batch testing cancelled")
        return
    
    print(f"\nStarting batch test - making {len(configs_to_test)} calls...")
    
    for i, config_key in enumerate(configs_to_test, 1):
        config = AMD_CONFIGURATIONS[config_key]
        
        print(f"\n[{i}/{len(configs_to_test)}] Testing: {config['name']}")
        print("-" * 50)
        
        call = make_amd_call(config_key)
        
        if call:
            results[config_key] = {
                'call_sid': call.sid,
                'status': call.status,
                'config': config
            }
            
            print(f"SUCCESS - Call SID: {call.sid}")
            
            # Just a brief pause to avoid overwhelming the console output
            if i < len(configs_to_test):
                print("   Moving to next configuration...")
                time.sleep(1)  # Just 1 second to space out console output
        else:
            results[config_key] = {
                'error': 'Failed to create call',
                'config': config
            }
            print(f"FAILED - Could not create call")
    
    # Summary
    print("\n" + "="*60)
    print("BATCH TEST SUMMARY")
    print("="*60)
    
    successful_calls = []
    failed_calls = []
    
    for config_key, result in results.items():
        config = result['config']
        print(f"\n{config['name']}:")
        if 'call_sid' in result:
            print(f"  SUCCESS - Call SID: {result['call_sid']}")
            print(f"  Status: {result['status']}")
            successful_calls.append(result['call_sid'])
        else:
            print(f"  FAILED - Error: {result.get('error', 'Unknown error')}")
            failed_calls.append(config_key)
    
    print(f"\n" + "="*60)
    print(f"BATCH TEST COMPLETED")
    print(f"Successful calls: {len(successful_calls)}")
    print(f"Failed calls: {len(failed_calls)}")
    print(f"="*60)
    
    if successful_calls:
        print(f"\nAll {len(successful_calls)} calls initiated successfully!")
        print(f"Each call is independent with its own AMD detection and webhook callbacks.")
        print(f"Check your webhook server logs for AMD results from each call.")
        print(f"\nCall SIDs for reference:")
        for i, call_sid in enumerate(successful_calls, 1):
            config_key = list(results.keys())[i-1]
            config_name = AMD_CONFIGURATIONS[config_key]['name']
            print(f"  {i}. {config_name[:30]}: {call_sid}")
        
        print(f"\nTo check results for any specific call:")
        print(f"   fetch_call_amd_results('CALL_SID')")
    
    if failed_calls:
        print(f"\nFailed configurations: {', '.join(failed_calls)}")
    
    print(f"\nNote: Calls are processed independently by Twilio.")
    print(f"AMD results will appear in webhook logs as each call completes.")

def quick_test_single_config(config_key):
    """Quick test of a single configuration"""
    
    if config_key not in AMD_CONFIGURATIONS:
        print(f"Error: Configuration '{config_key}' not found")
        print(f"Available: {list(AMD_CONFIGURATIONS.keys())}")
        return None
    
    print(f"\nQuick test: {config_key}")
    call = make_amd_call(config_key)
    
    if call:
        print(f"\nCall initiated: {call.sid}")
        print("Check webhook server logs for AMD results")
        print(f"Or use: fetch_call_amd_results('{call.sid}')")
    
    return call

In [12]:
# ==============================================================================
# 6. INTERACTIVE AMD TESTING MENU
# ==============================================================================

def amd_testing_menu():
    """Interactive menu for AMD testing"""
    
    while True:
        print("\n" + "="*60)
        print("TWILIO AMD TESTING MENU")
        print("="*60)
        print(f"From: {FROM_NUMBER}")
        print(f"To: {TO_NUMBER}")
        print(f"Webhook: {WEBHOOK_BASE_URL}")
        print("="*60)
        print("1. Test Single Configuration")
        print("2. Batch Test All Configurations") 
        print("3. Create Custom Configuration")
        print("4. Show Configuration Details")
        print("5. Fetch Call AMD Results")
        print("6. Monitor Call with AMD Results")
        print("7. Show Recent Calls")
        print("0. Exit")
        print("-"*60)
        
        choice = input("Select option (0-7): ").strip()
        
        if choice == "0":
            print("Exiting AMD Testing...")
            break
            
        elif choice == "1":
            print("\nAvailable AMD Configurations:")
            print("-" * 40)
            for key, config in AMD_CONFIGURATIONS.items():
                print(f"  {key}: {config['name']}")
                print(f"     {config['description']}")
            
            config_key = input("\nEnter configuration key: ").strip()
            if config_key:
                call = make_amd_call(config_key)
                if call:
                    monitor = input(f"\nMonitor call {call.sid}? (y/n): ").strip().lower()
                    if monitor == 'y':
                        monitor_call_with_amd_results(call.sid)
            
        elif choice == "2":
            batch_test_all_configurations()
            
        elif choice == "3":
            print("\nCreate Custom AMD Configuration")
            print("-" * 40)
            name = input("Configuration name: ").strip()
            description = input("Description: ").strip()
            
            print("\nAMD Parameters (press Enter for defaults):")
            timeout = input("Timeout (default 20s): ").strip() or "20"
            speech_threshold = input("Speech Threshold (default 2000ms): ").strip() or "2000"
            speech_end_threshold = input("Speech End Threshold (default 1200ms): ").strip() or "1200"
            silence_timeout = input("Silence Timeout (default 4000ms): ").strip() or "4000"
            
            detection_type = input("Detection Type (Enable/DetectMessageEnd, default Enable): ").strip() or "Enable"
            
            try:
                config_key = create_custom_config(
                    name, description, detection_type,
                    int(timeout), int(speech_threshold), 
                    int(speech_end_threshold), int(silence_timeout)
                )
                
                test_now = input(f"\nTest this configuration now? (y/n): ").strip().lower()
                if test_now == 'y':
                    call = make_amd_call(config_key)
                    if call:
                        monitor = input(f"\nMonitor call {call.sid}? (y/n): ").strip().lower()
                        if monitor == 'y':
                            monitor_call_with_amd_results(call.sid)
                            
            except ValueError:
                print("Invalid numeric values entered")
                
        elif choice == "4":
            print("\nAMD Configuration Details:")
            print("-" * 50)
            for key, config in AMD_CONFIGURATIONS.items():
                print(f"\n'{key}': {config['name']}")
                print(f"  Description: {config['description']}")
                params = config['params']
                print(f"  Detection: {params['machine_detection']}")
                print(f"  Timeout: {params['machine_detection_timeout']}s")
                print(f"  Speech Threshold: {params['machine_detection_speech_threshold']}ms")
                print(f"  Speech End Threshold: {params['machine_detection_speech_end_threshold']}ms")
                print(f"  Silence Timeout: {params['machine_detection_silence_timeout']}ms")
                
        elif choice == "5":
            call_sid = input("Enter Call SID: ").strip()
            if call_sid:
                fetch_call_amd_results(call_sid)
                
        elif choice == "6":
            call_sid = input("Enter Call SID: ").strip()
            if call_sid:
                monitor_call_with_amd_results(call_sid)
                
        elif choice == "7":
            try:
                print("\nFetching recent calls...")
                calls = client.calls.list(limit=10)
                
                if calls:
                    print(f"\nRecent Calls:")
                    print("-" * 80)
                    print(f"{'Call SID':<36} {'Status':<12} {'From':<15} {'To':<15} {'Date'}")
                    print("-" * 80)
                    
                    for call in calls:
                        date_str = call.date_created.strftime('%m/%d %H:%M') if call.date_created else 'N/A'
                        print(f"{call.sid:<36} {call.status:<12} "
                              f"{call.from_formatted or call.from_:<15} "
                              f"{call.to_formatted or call.to:<15} {date_str}")
                else:
                    print("No recent calls found")
                    
            except Exception as e:
                print(f"Error fetching calls: {str(e)}")
                
        else:
            print("Invalid choice! Please select 0-7.")

In [15]:
# ==============================================================================
# AMD TESTING MENU - Simple Interactive Interface
# ==============================================================================

def amd_testing_menu():
    """Simple menu to select and run AMD testing functions"""
    
    while True:
        print("\n" + "="*50)
        print("AMD TESTING MENU")
        print("="*50)
        print("1. Interactive AMD Testing")
        print("2. Batch Test All Configurations") 
        print("3. Quick Test Single Configuration")
        print("4. Make AMD Call")
        print("5. Fetch Call AMD Results")
        print("6. Monitor Call with AMD Results")
        print("0. Exit")
        print("-"*50)
        
        choice = input("Select option (0-6): ").strip()
        
        if choice == "0":
            print("Exiting...")
            break
            
        elif choice == "1":
            interactive_amd_testing()
            
        elif choice == "2":
            batch_test_all_configurations()
            
        elif choice == "3":
            print("\nAvailable configurations:")
            for key, config in AMD_CONFIGURATIONS.items():
                print(f"  {key}: {config['name']}")
            config_key = input("\nEnter configuration key: ").strip()
            if config_key:
                quick_test_single_config(config_key)
                
        elif choice == "4":
            print("\nAvailable configurations:")
            for key, config in AMD_CONFIGURATIONS.items():
                print(f"  {key}: {config['name']}")
            config_key = input("\nEnter configuration key: ").strip()
            if config_key:
                call = make_amd_call(config_key)
                if call:
                    monitor = input(f"\nMonitor call {call.sid}? (y/n): ").strip().lower()
                    if monitor == 'y':
                        monitor_call_with_amd_results(call.sid)
                        
        elif choice == "5":
            call_sid = input("Enter Call SID: ").strip()
            if call_sid:
                fetch_call_amd_results(call_sid)
                
        elif choice == "6":
            call_sid = input("Enter Call SID: ").strip()
            if call_sid:
                monitor_call_with_amd_results(call_sid)
                
        else:
            print("Invalid choice!")
            
        input("\nPress Enter to continue...")

# Run the menu
amd_testing_menu()


AMD TESTING MENU
1. Interactive AMD Testing
2. Batch Test All Configurations
3. Quick Test Single Configuration
4. Make AMD Call
5. Fetch Call AMD Results
6. Monitor Call with AMD Results
0. Exit
--------------------------------------------------

BATCH TESTING ALL AMD CONFIGURATIONS
Will test 6 configurations:
  - residential_fast: Residential/Mobile - Fast Detection
  - business_standard: Business - Standard Detection
  - voicemail_drop: Voicemail Message Drop
  - conservative: Conservative - High Accuracy
  - aggressive: Aggressive - Fast Response
  - custom_tuning: Custom Fine-Tuning

Starting batch test - making 6 calls...

[1/6] Testing: Residential/Mobile - Fast Detection
--------------------------------------------------

Making AMD call with: Residential/Mobile - Fast Detection
Description: Optimized for personal phones with short greetings
From: +551150266619
To: +551148583195

AMD Parameters:
  Detection Type: Enable
  Timeout: 15 seconds
  Speech Threshold: 1800 ms
  Speec

In [ ]:
# ==============================================================================
# 6. CLEANUP: Shut Down the Server and Tunnel
# ==============================================================================
# It's important to run this when you're done to close the public connection.
stop_server_and_ngrok(PUBLIC_URL)